In [2]:
from pyhealth.datasets import MIMIC3Dataset

mimic3_data = MIMIC3Dataset(root='../data/',
tables=["DIAGNOSES_ICD","PROCEDURES_ICD", "PRESCRIPTIONS"],
code_mapping = {'NDC': ("ATC", {"target_kwargs": {"level":3}})},
dev = True)

mimic3_data.stat()



Statistics of base dataset (dev=True):
	- Dataset: MIMIC3Dataset
	- Number of patients: 1000
	- Number of visits: 1295
	- Number of visits per patient: 1.2950
	- Number of events per visit in DIAGNOSES_ICD: 9.3544
	- Number of events per visit in PROCEDURES_ICD: 4.3351
	- Number of events per visit in PRESCRIPTIONS: 59.2556



'\nStatistics of base dataset (dev=True):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 1000\n\t- Number of visits: 1295\n\t- Number of visits per patient: 1.2950\n\t- Number of events per visit in DIAGNOSES_ICD: 9.3544\n\t- Number of events per visit in PROCEDURES_ICD: 4.3351\n\t- Number of events per visit in PRESCRIPTIONS: 59.2556\n'

In [3]:
mimic3_data.info()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



In [4]:
from pyhealth.tasks import drug_recommendation_mimic3_fn

mimic3_data = mimic3_data.set_task(task_fn=drug_recommendation_mimic3_fn)

mimic3_data.stat()

Generating samples for drug_recommendation_mimic3_fn: 100%|██████████| 1000/1000 [00:00<00:00, 9222.73it/s]


Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: drug_recommendation_mimic3_fn
	- Number of samples: 322
	- Number of patients: 115
	- Number of visits: 322
	- Number of visits per patient: 2.8000
	- conditions:
		- Number of conditions per sample: 44.5342
		- Number of unique conditions: 915
		- Distribution of conditions (Top-10): [('5856', 443), ('7100', 416), ('40301', 370), ('V5861', 316), ('V1251', 308), ('2875', 272), ('28521', 245), ('4280', 242), ('32723', 241), ('4019', 221)]
	- procedures:
		- Number of procedures per sample: 12.1118
		- Number of unique procedures: 303
		- Distribution of procedures (Top-10): [('3995', 490), ('3893', 299), ('9904', 284), ('9604', 135), ('966', 131), ('9671', 112), ('3895', 103), ('9390', 103), ('9907', 97), ('9672', 89)]
	- drugs:
		- Number of drugs per sample: 26.8882
		- Number of unique drugs: 165
		- Distribution of drugs (Top-10): [('B01A', 292), ('B05X', 287), ('A02B', 281), ('N02B', 279), ('A06A', 271), ('N02A', 251)

"Statistics of sample dataset:\n\t- Dataset: MIMIC3Dataset\n\t- Task: drug_recommendation_mimic3_fn\n\t- Number of samples: 322\n\t- Number of patients: 115\n\t- Number of visits: 322\n\t- Number of visits per patient: 2.8000\n\t- conditions:\n\t\t- Number of conditions per sample: 44.5342\n\t\t- Number of unique conditions: 915\n\t\t- Distribution of conditions (Top-10): [('5856', 443), ('7100', 416), ('40301', 370), ('V5861', 316), ('V1251', 308), ('2875', 272), ('28521', 245), ('4280', 242), ('32723', 241), ('4019', 221)]\n\t- procedures:\n\t\t- Number of procedures per sample: 12.1118\n\t\t- Number of unique procedures: 303\n\t\t- Distribution of procedures (Top-10): [('3995', 490), ('3893', 299), ('9904', 284), ('9604', 135), ('966', 131), ('9671', 112), ('3895', 103), ('9390', 103), ('9907', 97), ('9672', 89)]\n\t- drugs:\n\t\t- Number of drugs per sample: 26.8882\n\t\t- Number of unique drugs: 165\n\t\t- Distribution of drugs (Top-10): [('B01A', 292), ('B05X', 287), ('A02B', 281

In [11]:
from pyhealth.datasets import split_by_patient, get_dataloader
# from fractions import Fraction
from pyhealth.models import Transformer, GAMENet, RETAIN
import torch
import torch.optim as optim


train_data, eval_data, test_data = split_by_patient(mimic3_data, [.6,.2,.2], seed=1203)

train_dataloader = get_dataloader(train_data, batch_size=64, shuffle=True)
eval_dataloader = get_dataloader(eval_data, batch_size=64, shuffle=False)
test_dataloader = get_dataloader(test_data, batch_size=64, shuffle=False)

transformer_model = Transformer(
    dataset = mimic3_data,
    feature_keys = ["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
)

gamenet_model = GAMENet(
    dataset = mimic3_data
)

retain_model = RETAIN(dataset = mimic3_data,
    feature_keys = ["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",)

learning_rate = 5e-4

optimizers = {"Adam": optim.Adam,"SGD": optim.SGD,
"AdamW": optim.AdamW,"NAdam": optim.NAdam,
"RMSprop": optim.RMSprop}
models = {"Transformer": transformer_model, "GAMENet": gamenet_model, "RETAIN": retain_model}


In [12]:
from pyhealth.trainer import Trainer
from pyhealth.metrics.multilabel import multilabel_metrics_fn

scores = {}
metrics = {}
transformer_scores = {}
transformer_metrics = {}


for name, model in models.items():

    trainer = Trainer(model=model,enable_logging=False)
    if name == "Transformer":
        for opt_name, optimizer in optimizers.items():
            trainer.train(
            train_dataloader=train_dataloader,
            val_dataloader=eval_dataloader,
            epochs=100,
            optimizer_class=optimizer,
            monitor="pr_auc_samples")

            score = trainer.evaluate(test_dataloader)
            transformer_scores[opt_name] = score

            y_true, y_prob, loss = trainer.inference(test_dataloader)
            transformer_metrics[opt_name] = multilabel_metrics_fn(y_true, y_prob, metrics=["jaccard_samples","pr_auc_samples","f1_samples"])
    else:
        
        trainer.train(
            train_dataloader=train_dataloader,
            val_dataloader=eval_dataloader,
            epochs=100,
            monitor="pr_auc_samples"
        )

        score = trainer.evaluate(test_dataloader)
        scores[name] = score

        y_true, y_prob, loss = trainer.inference(test_dataloader)
        metrics[name] = multilabel_metrics_fn(y_true, y_prob, metrics=["jaccard_samples","pr_auc_samples","f1_samples"])

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(917, 128, padding_idx=0)
    (procedures): Embedding(305, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
          )
          (in

Epoch 0 / 100: 100%|██████████| 4/4 [00:00<00:00,  5.19it/s]

--- Train epoch-0, step-4 ---
loss: 1.0465



Evaluation: 100%|██████████| 2/2 [00:00<00:00, 28.19it/s]

--- Eval epoch-0, step-4 ---
pr_auc_samples: 0.2270
loss: 0.7325
New best pr_auc_samples score (0.2270) at epoch-0, step-4



...

--- Train epoch-99, step-400 ---
loss: 0.1987



Evaluation: 100%|██████████| 2/2 [00:00<00:00, 30.54it/s]

--- Eval epoch-99, step-400 ---


pr_auc_samples: 0.6568
loss: 0.2921


Evaluation: 100%|██████████| 1/1 [00:00<00:00, 19.80it/s]


In [13]:
for name in models.keys():
    if name == "Transformer":
        for opt_name in optimizers.keys():
            print(name)
            print(opt_name)
            print(transformer_scores[opt_name])
            print(transformer_metrics[opt_name])
    else:
        print(name)
        print(scores[name])
        print(metrics[name])
    

Transformer
Adam
{'pr_auc_samples': 0.6328098671739885, 'loss': 0.39127472043037415}
{'jaccard_samples': 0.38884732806001193, 'pr_auc_samples': 0.6328098671739885, 'f1_samples': 0.5529991736962544}
Transformer
SGD
{'pr_auc_samples': 0.6334469897232725, 'loss': 0.39125505089759827}
{'jaccard_samples': 0.38929723416513007, 'pr_auc_samples': 0.6334469897232725, 'f1_samples': 0.5536541573195907}
Transformer
AdamW
{'pr_auc_samples': 0.6448919604799548, 'loss': 0.45805150270462036}
{'jaccard_samples': 0.4013520016293195, 'pr_auc_samples': 0.6448919604799548, 'f1_samples': 0.564154396548941}
Transformer
NAdam
{'pr_auc_samples': 0.6526596446589905, 'loss': 0.4904535412788391}
{'jaccard_samples': 0.405263139947707, 'pr_auc_samples': 0.6526596446589905, 'f1_samples': 0.5687512666552972}
Transformer
RMSprop
{'pr_auc_samples': 0.6489714194947361, 'loss': 0.5396948456764221}
{'jaccard_samples': 0.38557558145550136, 'pr_auc_samples': 0.6489714194947361, 'f1_samples': 0.5484754445677129}
GAMENet
{'pr